### Model Parameter 설정

In [ ]:
#%pip install -q langchain langchain-openai

In [ ]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv(dotenv_path='../.env')

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

### 1) 모델 클래스 유형

#### LLM

In [ ]:
from langchain_openai import OpenAI

# model을 지정하지 않아도 실행되는 이유는 LangChain이 내부적으로 기본값을 설정함
llm = OpenAI()
print(llm.model_name)  # 기본 모델 확인

result = llm.invoke("한국의 대표적인 관광지 3군데를 추천해 주세요.")
print(type(result))
print(result)

#### ChatModel

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

#chat = ChatOpenAI(model="gpt-3.5-turbo-0125")
chat = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    temperature=0.7
)

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "이 시스템은 여행 전문가입니다."),
    ("human", "{user_input}"),
])

chain = chat_prompt | chat
result = chain.invoke({"user_input": "안녕하세요? 한국의 대표적인 관광지 3군데를 추천해 주세요."})

print(type(result))
print(result.content)

### 2) 모델 파라미터 설정
##### Temperature 효과
* 0.2 (낮은 값): 더 예측 가능하고 일반적인 이야기
* 1.2 (높은 값): 새로운 요소가 등장하며, 더 창의적이고 흥미로운 이야기 생성

##### Presence Penalty 효과
* 0.0 (낮은 값): 기존에 자주 등장하는 단어와 구조 유지
* 1.5 (높은 값): 새로운 표현과 독창적인 아이디어 등장

In [ ]:
from langchain_openai import ChatOpenAI

#  보수적인 설정 (일관된, 논리적인 이야기)
llm_before = ChatOpenAI(
    #model="gpt-3.5-turbo-0125",
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    temperature=0.2,  # 낮은 온도로 예측 가능한 출력
    presence_penalty=0.0,  # 기존 패턴 유지
    frequency_penalty=0.0,  # 반복 허용
    max_tokens=150,  # 출력 길이 제한
    top_p=1.0  # 확률 상위 100% 내에서 선택 (제한 없음)
)

#  창의적인 설정 (더 독창적이고 예측 불가능한 이야기)
llm_after = ChatOpenAI(
    #model="gpt-3.5-turbo-0125",
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    temperature=1.2,  # 높은 온도로 창의적인 답변 유도
    presence_penalty=1.2,  # 새로운 단어와 개념 유도
    frequency_penalty=0.5,  # 반복을 억제하여 더 다양한 표현 생성
    max_tokens=300,  # 더 긴 이야기 허용
    top_p=0.8  # 제한 없이 다양한 단어 선택 가능
)

# 질문 설정: 짧은 판타지 이야기 생성
# question = "마법의 세계에서 용이 인간과 친구가 되는 짧은 이야기를 써 주세요."
question = "마법의 세계에서 벌어지는 예상치 못한 사건을 주제로 독창적인 짧은 이야기를 만들어 주세요."

# 모델 호출
response_before = llm_before.invoke(question)
response_after = llm_after.invoke(question)

# 결과 출력
print(" Before (논리적이고 보수적인 이야기)")
print(response_before.content)

print("\n-------------------------\n")

print(" After (창의적인 이야기, 더 풍부한 표현)")
print(response_after.content)

#### Model의 종류에 따라 결과 값이 다름
* gpt-4o vs gpt-3.5-turbo-0125

In [ ]:
from langchain_openai import ChatOpenAI

# 변경된 설정 (더 창의적인 답변, 새로운 아이디어 유도)
#llm_after = ChatOpenAI(model="gpt-4o", temperature=1.0, presence_penalty=1.5)
#llm_after = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=1.0, presence_penalty=1.5)
llm_after = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=1.0,
    presence_penalty=1.5
)

# 질문 설정
question = "한국에서 가볼 만한 여행지를 추천해 주세요."

# 모델 호출
response_after = llm_after.invoke(question)

# 결과 출력
print("\n-------------------------\n")
print(" After (창의적인 응답, 새로운 아이디어 포함)")
print(response_after.content)

##### 2-1) 모델에 직접 파라미터를 전달 (모델 생성 시점)
##### Before / After 파라미터 차이
* temperature: Before(0.7) → 다양성 높은 추천 / After(0.3) → 정확한 일정 제공
* max_tokens: Before(300) → 간략한 응답 / After(800) → 세부 일정 포함
* frequency_penalty: Before(0.5) → 단어 반복 억제 / After(0.2) → 좀 더 자연스러운 응답
* presence_penalty: Before(0.5) → 다양한 장소 추천 / After(0.3) → 새로운 정보 포함

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

#  모델 파라미터 (Before: 기본적인 추천 / After: 맞춤형 세부 정보 추가)
before_params = {
    "temperature": 0.7,         # 랜덤성을 적절히 유지 (다양한 답변 유도)
    "max_tokens": 300,          # 응답 길이를 적절히 조절
    "frequency_penalty": 0.5,   # 반복 단어 억제
    "presence_penalty": 0.5,    # 새로운 단어 포함 장려
}

after_params = {
    "temperature": 0.3,         # 창의성을 낮추고 정확한 답변 유도
    "max_tokens": 800,          # 더 긴 답변을 생성 (세부 정보 포함)
    "top_p": 0.85,              # 확률 기반 샘플링 (일관성과 다양성 균형)
    "frequency_penalty": 0.2,   # 반복 단어 감소 (자연스러운 답변)
    "presence_penalty": 0.3,    # 새로운 정보 포함 장려
}

#  두 개의 모델 생성 (Before / After)
#before_model = ChatOpenAI(model="gpt-3.5-turbo-0125", **before_params)
before_model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    **before_params
)
#after_model = ChatOpenAI(model="gpt-3.5-turbo-0125", **after_params)
after_model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    **after_params
)

#  프롬프트 구성 (올바른 ChatMessagePromptTemplate 사용)
system_message = SystemMessagePromptTemplate.from_template(
    "당신은 여행 전문가입니다. 사용자의 요청에 맞는 최적의 여행지를 추천해 주세요."
)

user_message = HumanMessagePromptTemplate.from_template("{user_input}")

chat_prompt = ChatPromptTemplate.from_messages([system_message, user_message])

#  체인 생성 (Before / After)
before_chain = chat_prompt | before_model
after_chain = chat_prompt | after_model

#  질문 설정 (Before / After의 차이점을 비교할 수 있도록 변경)
test_question = "가족과 함께 3박 4일 동안 한국에서 여유롭게 여행할 수 있는 일정을 동선을 고려하여 자세하게 추천해 주세요."

#  Before 모델 실행
before_response = before_chain.invoke({"user_input": test_question})

#  After 모델 실행
after_response = after_chain.invoke({"user_input": test_question})

#  결과 출력
print(" [Before] 모델 결과")
print(before_response.content)
print("\n" + "="*80 + "\n")  # 가독성을 위한 구분선
print(" [After] 모델 결과")
print(after_response.content)

#### 2-2) 모델에 추가적인 파라미터를 전달
- bind() 메서드를 사용
- max_tokens=50 (기본) / max_tokens=150 (상세한 설명) 
- stop=["."] → 첫 번째 마침표에서 응답을 중단
- temperature=0.8 → 보다 창의적인 답변을 생성함

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

#  프롬프트 설정 (천문학 질문에 대한 답변을 생성하는 시스템)
prompt = ChatPromptTemplate.from_messages([
    ("system", "이 시스템은 천문학 질문에 대해 명확하고 자세한 답변을 제공할 수 있습니다."),
    ("user", "{user_input}"),
])

#  기본 모델 설정 (기본적인 답변 생성)
#base_model = ChatOpenAI(model="gpt-3.5-turbo-0125", max_tokens=150)  # 150 토큰 제한
base_model = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    max_tokens=150
)

#  질문 설정
# 1. MAX_TOKENS 차이를 보여주는 질문 (길이 제한 효과)
max_tokens_question = "인공지능의 발전이 미래 사회에 미칠 영향을 긍정적 측면과 부정적 측면으로 나누어 자세히 설명해 주세요."

# 2. STOP 파라미터 차이를 보여주는 질문 (중단점 효과)
stop_question = "Python 프로그래밍을 배우는 초보자에게 추천하는 학습 단계를 순서대로 설명해 주세요. 각 단계별로 구체적인 방법과 팁을 포함해서 답변해 주세요."

# 3. TEMPERATURE 차이를 보여주는 질문 (창의성 vs 정확성)
temperature_question = "시간 여행이 가능하다면 과거의 어느 시대로 가고 싶은지와 그 이유를 창의적으로 설명해 주세요."

# 4. 복합적 비교를 위한 질문 (모든 파라미터 효과)
complex_question = "화성에 인류가 정착하기 위해 필요한 기술과 준비사항들을 단계별로 설명하고, 각 단계에서 예상되는 도전과제와 해결방안을 제시해 주세요."


question = stop_question

#  Before (기본 max_tokens=150)
messages = prompt.format_messages(user_input=question)
before_answer = base_model.invoke(messages)

#  Before 출력
print("\n [Before] 기본 max_tokens=150 (기본 답변)")
print(before_answer.content)

print("\n" + "="*100 + "\n")  # 가독성을 위한 구분선

#  After (파라미터 조정 후 비교)
stop_chain = prompt | base_model.bind(max_tokens=150, stop=["."])  # 첫 번째 마침표에서 중단
temp_chain = prompt | base_model.bind(max_tokens=150, temperature=0.8)  # 창의적인 답변 유도

stop_answer = stop_chain.invoke({"user_input": question})
temp_answer = temp_chain.invoke({"user_input": question})

#  After 출력 (stop vs temperature 비교)
print(" [After] max_tokens=150, stop=['.'] (첫 번째 마침표에서 응답 중단)")
print(stop_answer.content)

print("\n" + "="*100 + "\n")  # 가독성을 위한 구분선

print(" [After] max_tokens=150, temperature=0.8 (창의적인 답변)")
print(temp_answer.content)